In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=1
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i
    
# get the initial h from fluid to solid, for applying bd for first
# iteration in solid, done offline
h_f2s_init = {
    1: 39171.64389847,
    5: 7834.32877969,
    20: 1958.58219492
}

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_rr.cmd' % k)
solverF.init_grid()
solverF.init_robin_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFRSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFRSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag, init_t=300.0, init_h=4.0*kappa/k)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
abTempS = InterfaceData(size=inodes, value=T_bot)
abTempS2F = InterfaceData(size=inodes, value=T_bot)

TempF = InterfaceData(size=inodes, value=300.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hF2S = hF[f2s]
hS = solverS.get_interface_h(tag=interface_tag)
hS2F = hS[s2f]

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FRSR%i.log' % k, mode='w')
flog.write('Fluid Robin with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FRSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        TempF.backup()
        abTempS2F.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # map
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid interface
        solverS.update_robin_interface(
                tag=interface_tag,
                Ta=abTempF2S.curr,
                h_coeff=hF2S
            )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid ambient temperature
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # mapping
        abTempS2F.curr[:] = abTempS.curr[s2f]
        
        # update residual
        TempF.update_res()
        abTempS2F.update_res()
        
        TempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(TempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(abTempS2F)
            under_relax.update_solution(abTempS2F)
            solverF.put_robin_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                t_inf=abTempS2F.curr,
                h_coeff=hS2F
            )
            solverF.restore_previous_solutions()
            solverS.restore()

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=1.
step=1, pc_iterations=1.
step=2, pc_iterations=1.
step=3, pc_iterations=1.
step=4, pc_iterations=1.
step=5, pc_iterations=1.
step=6, pc_iterations=1.
step=7, pc_iterations=1.
step=8, pc_iterations=1.
step=9, pc_iterations=1.
step=10, pc_iterations=1.
step=11, pc_iterations=1.
step=12, pc_iterations=1.
step=13, pc_iterations=1.
step=14, pc_iterations=1.
step=15, pc_iterations=1.
step=16, pc_iterations=1.
step=17, pc_iterations=1.
step=18, pc_iterations=1.
step=19, pc_iterations=1.
step=20, pc_iterations=1.
step=21, pc_iterations=1.
step=22, pc_iterations=1.
step=23, pc_iterations=1.
step=24, pc_iterations=1.
step=25, pc_iterations=1.
step=26, pc_iterations=1.
step=27, pc_iterations=1.
step=28, pc_iterations=1.
step=29, pc_iterations=1.
step=30, pc_iterations=1.
step=31, pc_iterations=1.
step=32, pc_iterations=1.
step=33, pc_iterations=1.
step=34, pc_iterations=1.
step=35, pc_iterations=1.
step=36, pc_iterations=1.
step=37, pc_iterations=1.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFRSR%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.22399567 0.25036423 0.27042595 0.28679901 0.30062697 0.31254491
 0.32295781 0.33214539 0.34031234 0.34761418 0.35417245 0.36008408
 0.36542768 0.37026804 0.37465937 0.37864766 0.3822725  0.38556832
 0.38856544 0.39129073 0.39376821 0.39601948 0.39806406 0.39991966
 0.40160241 0.40312704 0.40450704 0.4057548  0.40688167 0.40789814
 0.40881383 0.4096376  0.41037764 0.41104148 0.41163603 0.4121677
 0.41264233 0.41306532 0.41344161 0.41377572 0.41407179 0.41433359
 0.41456455 0.4147678  0.41494615 0.41510214 0.41523805 0.41535591
 0.41545752 0.41554446 0.4156181  0.41567959 0.41572993 0.41576988
 0.41580005 0.41582086 0.41583252 0.41583509 0.4158284  0.4158121
 0.41578561 0.41574813 0.4156986  0.41563571 0.41555783 0.415463
 0.4153489  0.41521281 0.41505153 0.41486134 0.41463795 0.4143764
 0.41407096 0.41371506 0.41330116 0.4128206  0.41226343 0.41161828
 0.41087205 0.41000973 0.40901406 0.40786514 0.40654002 0.40501215
 0.40325068 0.40121969 0.39887707 0.39617324 0.39304939 0.38943512
